In [7]:

import os
import pandas as pd
import json
import nbformat
from tqdm import tqdm
import time
from datetime import datetime

In [8]:
def get_ipynb_filenames(directory_path):
    ipynb_files = []

    # Walk through directory and subdirectories
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.ipynb'):
                # Get the full path of the file or just the file name
                ipynb_files.append(os.path.join(root, file))

    return ipynb_files

In [9]:
def get_filenames_with_endsWithSubString(directory_path, endsWithSubString):
    input_files = []

    # Walk through directory and subdirectories
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith(endsWithSubString):
                # Get the full path of the file or just the file name
                input_files.append(os.path.join(root, file))

    return input_files

Read notebook code cells

In [10]:
def read_code_cells_with_numbers(notebook_path):
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = json.load(f)

    code_cells = []

    for i, cell in enumerate(notebook['cells']):
        if cell['cell_type'] == 'code':
            # Combine the source lines into a single string for each code cell
            code = ''.join(cell['source'])
            # Store the cell number and code as a tuple
            code_cells.append((i+1, code))  # i+1 to make the cell number 1-based

    return code_cells

In [11]:
def read_all_cells(notebook_path):
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = json.load(f)

    nb_cells = []

    for i, cell in enumerate(notebook['cells']):
            # Combine the source lines into a single string for each code cell
            code = ''.join(cell['source'])
            # Store the cell number and code as a tuple
            nb_cells.append((i+1, code, cell['cell_type']))  # i+1 to make the cell number 1-based
            
            

    return nb_cells

In [14]:
directory_path = '/home/jupyter-saeed3/Dataset/kgtorrent_sample_384/'
ipynb_files = get_ipynb_filenames(directory_path)
len(ipynb_files)

372

# Link Mistral API

In [15]:
import requests
import json

# Replace 'YOUR_API_KEY' with the API key you generated
API_KEY = "FsYbhKyXBjQeontjJFOdjVACgtH0ljEN"

API_URL = "https://codestral.mistral.ai/v1/fim/completions"

In [16]:
def asking_to_codestral(asking_request, model="codestral-latest"):
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

    # Prompt to instruct the model to generate a description
    # prompt = f"Describe the following code snippet:\n\n{code_snippet}\n\nDescription:"

    payload = {
        "model": model,
        "prompt": asking_request,
        "temperature": 0.7  # Adjust for randomness
        # "max_tokens": 70    # Limit the response length
    }

    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        # return result
        answer = result["choices"][0]["message"]["content"]
        return answer
        # return result.get("choices", [{}])[0].get("text", "No description generated.")
    except requests.exceptions.RequestException as e:
        return f"Error: {e}"


In [24]:
code_snippet = """def factorial(n):
if n == 0:
    return 1
else:
    return n * factorial(n - 1)"""

description = asking_to_codestral("describe this code snippet in a sentence. " + code_snippet)
print("Generated Description:")
print(description)

Generated Description:


The code snippet presents a recursive function named 'factorial' that calculates the factorial of a given non-negative integer 'n'. The function checks if 'n' equals 0, in which case it returns 1, otherwise it multiplies 'n' with the factorial of 'n-1' until it reaches the base case (n=0).


# Ask Code Stral to get the promt for the code cell

In [17]:
directory_path = '/home/jupyter-saeed3/Dataset/kgtorrent_sample_384/'
output_path = '/home/jupyter-saeed3/Dataset/kgtorrent_384_prompt/'

# output_path = '/home/jupyter-saeed3/LLM_chat_conversion/'


ipynb_files = get_ipynb_filenames(directory_path)
len(ipynb_files)

372

provide earlier code to the model when asking to generate the prompt of the current cell. 

In [21]:
for nb_file_name, iterate in zip(ipynb_files[200:], tqdm(range(172))):
    output_file = nb_file_name.replace('.ipynb','_codestral.csv').split('/')[-1]
    output_file = output_path + output_file 

    formatted_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    time.sleep(0.05)
    
    try:
        # nb_cells = read_all_cells(nb_file_name)
        nb_cells = read_code_cells_with_numbers(nb_file_name)
        previous_code_cells = ''
        
        cell_number_list, cell_type_list, prompt_list, cell_content_list = [], [], [], []

        for cell_number, current_code_cell in nb_cells:
            if len(current_code_cell) > 0:
                               
                asking = "What specific prompt should be used to generate the following code as the model's output? The answer should be a like a prompt which further be asked by an user. DO NOT generate extra texts. Provide precise answer.  Here is the code: \n" + current_code_cell + "\n\n\n\n Here is the previous part of that code snippet. Use this to understand what happerns earlier in this notebook. " +  previous_code_cells
                
                previous_code_cells = previous_code_cells + current_code_cell

                response = asking_to_codestral(asking)

                # response_corrected = asking_to_codestral("Its a response from your model, but I need a precise answer. The answer should be a like a prompt which further be asked by an user. Here is the original response \n " + response) 
                
                prompt_list.append(response)
                cell_content_list.append(current_code_cell)
                cell_number_list.append(cell_number)
                # cell_type_list.append(cell_type)


        df = pd.DataFrame({'cell_number':cell_number_list, 'prompt': prompt_list, 'cell_content': cell_content_list})
        df.to_csv(output_file)
        
        with open("log_sample_gh_codestral.txt", "a") as file:
            file.write(formatted_time + '  done   '  + output_file + '\n')
        
    except Exception as e:
        
        with open("log_sample_gh_codestral.txt", "a") as file:
            file.write(formatted_time + '  fail   '  + output_file + str(e) + '\n')
        pass

print ('codestral prompt generation complete')

 99%|█████████▉| 171/172 [17:06<00:06,  6.00s/it]

codestral prompt generation complete


In [6]:
print ('codestral prompt generation complete')

codestral


### Read compare csvs

In [70]:
compare_csvs = get_filenames_with_endsWithSubString(input_file_directory_with_prompt, "Compare_bert.csv")
len(compare_csvs)


5432

In [72]:
compare_csvs[0]

'/home/jupyter-saeed3/LLM_chat_conversion/generated_prompt/nb_gh_sample/nb_393848_promptCompare_bert.csv'

In [76]:
df = pd.read_csv(compare_csvs[0])
df

,Unnamed: 0,cell_number,similarity,gpt35terbo_prompt,gpt4oMini_prompt,cell_content
0,0,1,0.526510,Prompt: \nExplain how the autoreload extension...,Generate a code snippet that includes the foll...,%load_ext autoreload\n%autoreload 2
1,1,2,0.782681,Please generate the code snippet for setting u...,Generate Python code that sets up matplotlib f...,import matplotlib as mpl\nfrom matplotlib impo...
2,2,3,0.787148,Please write a prompt that would lead to the i...,Generate a Python code snippet that imports th...,import numpy as np\nimport random as rn
3,3,4,0.707787,Prompt: Import the necessary functions from th...,Generate a Python code snippet that imports th...,"from scipy.spatial.distance import pdist,cdist..."
4,4,5,0.691481,Provide a prompt that would lead to the creati...,Generate a Python function named `center_axis`...,"def center_axis(ax,lim=1):\n ax.axis([-lim,..."
5,5,6,0.598082,Please write a prompt that will generate code ...,Generate a Python code snippet that creates tw...,"Xp = [[-1.5,-1.5],\n [-1.0,-0.5],\n ..."
6,6,7,0.510788,Create a scatter plot with two different sets ...,Generate a Python code snippet using Matplotli...,"fig,ax = plt.subplots(1,1,subplot_kw={'aspect'..."
7,7,8,0.176769,What is the prompt that should be used to gene...,Generate a Python code snippet that creates a ...,"fig,ax = plt.subplots(1,1,subplot_kw={'aspect'..."
8,8,9,0.778508,Please write a prompt that will generate code ...,Generate a code snippet that concatenates two ...,"X = np.concatenate((Xp,Xn),axis=0)\nX = np.ins..."
9,9,10,0.582609,Create a prompt that initializes an exponentia...,Generate a Python code snippet that imports a ...,import helper as he\nfrom cd.model import util...


In [ ]:
for compare_csv in compare_csvs[:100]:
    compare_df = pd.read_csv(compare_csv)
    
    count_greater_than_0_7 = (compare_df['similarity'] > 0.7).sum()
    total_count = compare_df['similarity'].size
    
    print (" Similarity ", count_greater_than_0_7 * 100 / total_count , " % ")
    

In [49]:
! pip install transformers scipy torch


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [61]:
# prompt1 = "What should be the prompt for the code snippet that prints \"So far we talked about big data. What about structured big data?\" and creates a SQLContext object?"
# prompt2 = "Generate a code snippet in Python that demonstrates the use of PySpark to create an SQLContext and print a message about structured big data."

# prompt1 = "Write a Python script that imports necessary libraries for data processing and analysis, including time, numpy, operator, pyspark, and json."
# prompt2 = "Generate a Python code snippet that includes the following imports: `time`, `numpy`, `operator`, `SQLContext`, `Row`, `types` from `pyspark.sql`, `functions` from `pyspark.sql`, and `SparkContext`, as well as `json`."

prompt1 = "What is the time taken to filter out elements in a file RDD that have a length less than 2 and collect the results?"
prompt2 = "Generate a Python code snippet that filters an RDD based on the length of its elements, measures the time taken for the operation, and prints the filtered results along with the time taken."




In [62]:
from transformers import AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine
import torch

# Load the tokenizer and model for Sentence-BERT
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to get embeddings from Sentence-BERT and ensure they are 1-D
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    # Get mean across token embeddings and flatten to ensure 1-D vector
    return outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

# Prompts to compare
# prompt1 = "Write a Python script that imports necessary libraries for data processing and analysis, including time, numpy, operator, pyspark, and json."
# prompt2 = "Generate a Python code snippet that includes the following imports: `time`, `numpy`, `operator`, `SQLContext`, `Row`, `types` from `pyspark.sql`, `functions` from `pyspark.sql`, and `SparkContext`, as well as `json`."

# Get embeddings
embedding1 = get_embedding(prompt1)
embedding2 = get_embedding(prompt2)

# Calculate cosine similarity
similarity = 1 - cosine(embedding1, embedding2)
print(f"Semantic Similarity: {similarity:.2f}")


Semantic Similarity: 0.74


In [83]:
kgtorrent_path = "/storage/saeed/nb_data_kgtorrent/draft/"


In [86]:
filename = "kerneler_starter-savenew2-366176d6-f.ipynb"
output_filename = "kerneler_starter-savenew2-366176d6-f.csv"

with open(kgtorrent_path+filename, 'r', encoding='utf-8') as f:
    notebook = nbformat.read(f, as_version=4)

# Extract cells and their information
data = []
for cell in notebook['cells']:
    cell_type = cell['cell_type']
    source = cell['source']
    output = None

    # Handle code cell outputs
    if cell_type == 'code' and 'outputs' in cell:
        output = '\n'.join(
            output['text'] if 'text' in output else str(output)
            for output in cell['outputs']
        )

    # Append cell data to the list
    data.append({
        'Cell Type': cell_type,
        'Source': source,
        'Output': output
    })

# Convert to DataFrame
df = pd.DataFrame(data)

# Write to CSV
csv_path = kgtorrent_path+output_filename
df.to_csv(csv_path, index=False)

print(f"Notebook content has been written to {csv_path}")

PermissionError: [Errno 13] Permission denied: '/storage/saeed/nb_data_kgtorrent/draft/kerneler_starter-savenew2-366176d6-f.csv'

In [46]:
!pip install rouge_score

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f486910ec29c16dad8a339247d0212337ee2331b1f1675bb3264392d6608cad6
  Stored in directory: /home/jupyter-saeed3/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [48]:
from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Define the explanations from two LLMs
explanation_1 = "What is the time taken to map elements in an RDD and calculate the time difference between starting and ending the operation?."
explanation_2 = "Generate a code snippet in Python that measures the time taken to map an RDD by multiplying each element by 2, and prints the time taken in seconds."

# Compute ROUGE-L score
score = scorer.score(explanation_1[0], explanation_2[0])

print("ROUGE-L Score:", score['rougeL'].fmeasure)


ROUGE-L Score: 0.0


In [28]:
def read_all_cells_for_fail_check(notebook_path):
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = json.load(f)
    
    print(notebook)
    nb_cells = []

    for i, cell in enumerate(notebook['worksheets']['cells']):
            # Combine the source lines into a single string for each code cell
            code = ''.join(cell['source'])
            # print (code)
            # Store the cell number and code as a tuple
            nb_cells.append((i+1, code, cell['cell_type']))  # i+1 to make the cell number 1-based
            
    return nb_cells

In [25]:
# fail_filename = 'nb_gh_sample/nb_1107626.ipynb'
fail_filename = 'nb_gh_sample/nb_1107929.ipynb'


try:
    nb_cells = read_all_cells_for_fail_check(fail_filename)
    print (nb_cells)
    
#     cell_number_list, cell_type_list, prompt_list, cell_content_list = [], [], [], []

#     for cell_number, content, cell_type in nb_cells:
#         if len(content) > 0:
#             messages = "What specific prompt should I use to generate this code as the model's output?  Here is the code: \n\n " + content
#             response = chat_with_gpt(messages)

#             prompt_list.append(response)
#             cell_content_list.append(content)
#             cell_number_list.append(cell_number)
#             cell_type_list.append(cell_type)


#     df = pd.DataFrame({'cell_number':cell_number_list, 'cell_type':cell_type_list, 'prompt': prompt_list, 'cell_content': cell_content_list})
    # Print or process the code cells
except Exception as e:
    print (f"Fail {'-'*5} {fail_filename+'.csv'}{'-'*10}")
    pass

{'cells': [{'cell_type': 'markdown', 'metadata': {}, 'source': ['### Target\n', '\n', '![](http://r-statistics.co/screenshots/ggplot_masterlist_8_1.png)']}, {'cell_type': 'code', 'execution_count': 1, 'metadata': {'collapsed': True}, 'outputs': [], 'source': ['import pandas as pd\n', 'mpg= pd.read_csv("http://goo.gl/uEeRGu") ']}, {'cell_type': 'code', 'execution_count': 10, 'metadata': {}, 'outputs': [{'data': {'text/html': ['<div class="vega-embed" id="88dbf520-9c34-4cf2-b36b-268cfb47b1b0"></div>\n', '\n', '<style>\n', '.vega-embed svg, .vega-embed canvas {\n', '  border: 1px dotted gray;\n', '}\n', '\n', '.vega-embed .vega-actions a {\n', '  margin-right: 6px;\n', '}\n', '</style>\n']}, 'metadata': {'jupyter-vega': '#88dbf520-9c34-4cf2-b36b-268cfb47b1b0'}, 'output_type': 'display_data'}, {'data': {'application/javascript': ['var spec = {"data": {"values": [{"displ": 1.8, "hwy": 29, "cty": 18, "drv": "f", "year": 1999, "model": "a4", "trans": "auto(l5)", "fl": "p", "class": "compact",

In [29]:
fail_filename = 'nb_gh_sample/nb_1107626.ipynb'
# fail_filename = 'nb_gh_sample/nb_1107929.ipynb'

try:
    nb_cells = read_all_cells_for_fail_check(fail_filename)
    print (nb_cells)
    # Print or process the code cells
except Exception as e:
    print (f"Fail {'-'*5} {fail_filename+'.csv'}{'-'*10}")
    pass

{'metadata': {'name': '02B_feature_extraction'}, 'nbformat': 3, 'nbformat_minor': 0, 'worksheets': [{'cells': [{'cell_type': 'heading', 'level': 1, 'metadata': {}, 'source': ['Feature Extraction']}, {'cell_type': 'markdown', 'metadata': {}, 'source': ['Here we will talk about an important piece of machine learning: the extraction of\n', 'quantitative features from data.  By the end of this section you will\n', '\n', '- Know how features are extracted from real-world data.\n', '- See an example of extracting numerical features from textual data\n', '\n', 'In addition, we will go over several basic tools within scikit-learn which can be used to accomplish the above tasks.']}, {'cell_type': 'heading', 'level': 2, 'metadata': {}, 'source': ['What Are Features?']}, {'cell_type': 'heading', 'level': 3, 'metadata': {}, 'source': ['Numerical Features']}, {'cell_type': 'markdown', 'metadata': {}, 'source': ['Recall that data in scikit-learn is expected to be in two-dimensional arrays, of size\n